In [1]:
import turicreate as gl
import tensorflow as tf
import matplotlib.pyplot as plt
from turicreate_cross_validation.cross_validation import shuffle_sframe, StratifiedKFold, cross_val_score

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/ops.py:875: DeprecationWarning: builtin type EagerTensor has no __module__ attribute
  EagerTensor = c_api.TFE_Py_InitEagerTensor(_EagerTensorBase)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


In [2]:
SF = gl.SFrame.read_csv('/home/test-machine/training.csv',verbose=False)
print ("Done reading")
SF.head

Done reading


<bound method SFrame.head of Columns:
	APL	float
	AvgPktPerSec	float
	IAT	float
	NumForward	int
	BytesEx	int
	BitsPerSec	float
	NumPackets	int
	StdDevLen	float
	Duration	int
	IOPR	float
	isBot	int

Rows: 356156

Data:
+---------+--------------+------------+------------+---------+------------+------------+
|   APL   | AvgPktPerSec |    IAT     | NumForward | BytesEx | BitsPerSec | NumPackets |
+---------+--------------+------------+------------+---------+------------+------------+
|   0.0   |   0.673102   | 1782790.0  |     6      |    0    |    0.0     |     6      |
|   0.0   |   0.124399   | 16100000.0 |     2      |    0    |    0.0     |     2      |
|   0.0   |  0.0208823   | 95800000.0 |     2      |    0    |    0.0     |     2      |
| 119.714 |   80.6889    |  14458.8   |     4      |   838   |  77276.9   |     7      |
|   0.0   |   11111.1    |   180.0    |     2      |    0    |    0.0     |     2      |
|   0.0   |   0.665698   | 1802620.0  |     6      |    0    |    0.0 

In [3]:
SF.shape

(356156, 11)

In [17]:
myfeatures = ['APL','AvgPktPerSec','IAT','NumForward','BytesEx','BitsPerSec','NumPackets','StdDevLen','Duration','IOPR']

model = gl.random_forest_classifier.create(SF,features=myfeatures,target='isBot',verbose = False)

In [18]:
Test = gl.SFrame.read_csv('/home/test-machine/testing.csv',verbose=False)
Test.head

<bound method SFrame.head of Columns:
	APL	float
	AvgPktPerSec	float
	IAT	float
	NumForward	int
	BytesEx	int
	BitsPerSec	float
	NumPackets	int
	StdDevLen	float
	Duration	int
	IOPR	float
	isBot	int

Rows: 309187

Data:
+---------+--------------+-----------+------------+---------+------------+------------+
|   APL   | AvgPktPerSec |    IAT    | NumForward | BytesEx | BitsPerSec | NumPackets |
+---------+--------------+-----------+------------+---------+------------+------------+
|   0.0   |   0.210342   | 5150350.0 |     13     |    0    |    0.0     |     13     |
|   91.4  |   1416.53    |  743.105  |     10     |   1828  | 1035770.0  |     20     |
|  36.75  |   196.078    |  5828.57  |     4      |   294   |  57647.1   |     8      |
|  36.75  |   181.818    |  6285.71  |     4      |   294   |  53454.5   |     8      |
| 110.438 |   567.346    |  1819.45  |     16     |   3534  |  501250.0  |     32     |
|   18.0  |   6015.04    |  221.667  |     2      |    72   |  866165.0  |    

In [23]:
import time
start_time = time.time()
temp = list(model.predict(Test))
end_time = time.time()
print ('end-time ',end_time)
print ('start_timee ',start_time)
print ('end-time - start_time ',(end_time - start_time)/309187)
print (gl.toolkits.evaluation.confusion_matrix(Test['isBot'],gl.SArray(temp)))

end-time  1527117934.5770438
start_timee  1527117934.224791
end-time - start_time  1.1392869745057166e-06
+--------------+-----------------+--------+
| target_label | predicted_label | count  |
+--------------+-----------------+--------+
|      1       |        1        | 142720 |
|      0       |        1        | 46181  |
|      0       |        0        | 69665  |
|      1       |        0        | 50621  |
+--------------+-----------------+--------+
[4 rows x 3 columns]



In [24]:
print (gl.toolkits.evaluation.accuracy(Test['isBot'],gl.SArray(temp)))

0.6869143916141365


In [25]:
print (gl.toolkits.evaluation.precision(Test['isBot'],gl.SArray(temp)))

0.7555280279087988
